In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(tot_pages):
    for page in range(1,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       

        url_main = f"https://www.sportdoma.ru/catalog/trenazher/powergym/{page}.html?sort=populartop&type=11&marka=all&seria=all&all="
        response = requests.get(url_main, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_ = 'name')
        for item in data:
            if item.find('a') != None:
                if bool(re.search(pattern = re.escape('catalog'), string = item.find('a').attrs['href'])):
                    item_url = 'https://www.sportdoma.ru' + item.find('a').attrs['href']
                    yield item_url

In [3]:
def get_item(category_gen, tot_pages):
    count = 0
    for url_item in get_url(tot_pages):
        count += 1
        response = requests.get(url_item, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
        except:
            title = ''

        try:
            article = soup.find(class_ = 'art').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '')
        except:
            article = ''

        try:
            image = soup.find(class_ = 'big_foto')
            url = 'https://www.sportdoma.ru' + image.find('a').attrs['href']
            image_ref = f"images/{category_gen}/{count}_sportdoma_{article}.jpeg"
            with open(
                f"images/{category_gen}/{count}_sportdoma_{article}.jpeg", "wb"
            ) as f:
                r = requests.get(url)
                f.write(r.content)   

        except:
            print(f'Log: no image for {article} found...')
            url = ''
            image_ref = ""

        try:
            info_lst = []
            info = soup.find_all(class_ = 'text_block3')
            for item in info:
                text = item.text.replace('\n', '').strip()
                text = re.sub(r"(?<=\w)([А-Я])", r" \1", text)
                info_lst.append(text.strip())

            desc = info_lst[1]
            char = info_lst[0]
        except:
            char = ''
            desc = ''

        try:
            price = soup.find(class_ = 'number').text
            price = price.replace('Р.', '').replace(' ', '')

        except:
            price = ''

        try:
            cat = soup.find(id = 'breadcramps').text
            temp_str = re.sub(re.compile('[^а-яА-Я ]'), '', cat)
            lst = re.findall('[А-Я][^А-Я]*', temp_str)

            try:
                cat_1 = lst[-2]
            except:
                cat_1 = ''
            try:
                cat_2 = lst[-1]
            except:
                cat_2 = ''
            try:
                cat_3 = re.sub(re.compile('[^a-zA-Z ]'), '', cat).strip()
            except:
                cat_3 = ''
        except:
            cat_1 = ''
            cat_2 = ''
            cat_3 = ''

        yield title, article, price, cat_1, cat_2, cat_3, url, desc, char, image_ref

In [4]:
df_columns = [
    "title",
    "article",
    "price",
    "cat_1",
    "cat_2",
    "cat_3",
    "url",
    "description",
    "caracteristics",
    "img_ref",
]

df = pd.DataFrame(columns = df_columns)
save_frequency = 200
log_frequency = 5

In [5]:
import time
def to_csv(category_gen, tot_pages):

    counter = 0  

    for item in get_item(category_gen, tot_pages):
        data = []
        for i in range(len(df_columns)):
            data.append(item[i])
        df.loc[len(df.index)] = data
        time.sleep(0.05)

        counter += 1
        if counter % save_frequency == 0:
            df.to_csv(f"{category_gen}_sportdoma.csv")
            print(f'{counter} items saved in csv..')

    df.to_csv(f"{category_gen}_sportdoma.csv")


    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv("trenazhery_na_svobodnykh_vesakh", 7)



Log: no image for 20521 found...
Log: no image for 20574 found...
Log: no image for 20578 found...
Log: no image for 20656 found...
Log: no image for 20658 found...
Log: no image for 20659 found...
Log: no image for 20660 found...
Log: no image for 20662 found...
Log: no image for 20663 found...
Log: no image for 20664 found...
Log: no image for 21227 found...
Log: no image for 21232 found...
Log: no image for 21233 found...
Log: no image for 21411 found...
Log: no image for 21858 found...
Log: no image for 21859 found...
Log: no image for 21860 found...
Log: no image for 21861 found...
Log: no image for 21862 found...
Log: no image for 21863 found...
Log: no image for 21864 found...
Log: no image for 21865 found...
Log: no image for 21866 found...
Log: no image for 21867 found...
Log: no image for 21868 found...
Log: no image for 21869 found...
Log: no image for 21870 found...
Log: no image for 21871 found...
Log: no image for 21872 found...
Log: no image for 21873 found...
Log: no im